# Integrated Bioinformatics Project : Flux_Balance_F_Prausnitzii 
#### Students: Michael Shawn Neilsson, Biancamaria FLorenzi, Prabhat Juyal, Tim Blokker
#### Supervisors: Clémence Joseph, Karoline Faust

In [1]:
#import packages
from libchebipy import ChebiEntity
import json
import requests
import molmass
import io
import cobra
import functions
import re
from functions import *
import cobrababel
import pandas as pd

In [2]:
#model = cobra.io.read_sbml_model('Data/f_prausnitzii.xml')
#info(model)
model=cobra.io.load_matlab_model('Data/iFpraus_v_1_0.mat')
info(model)


Metabolites :  833
Reactions :  1030
Genes :  602
Compartments :  {'c': '', 'e': ''}
Objective function : 
 Maximize
1.0*Biomass_FP - 1.0*Biomass_FP_reverse_ee33b 



In [3]:
x=[model.reactions.get_by_id(compound_id).name for compound_id in model.medium]
x=[compound[:(len(compound)-3)] for compound in x]
x=[compoundname.replace("EX_", "") for compoundname in x]
x=[compoundname.replace("excha", "").strip() for compoundname in x]
#";".join(x)

In [4]:
mapped_compounds=Mol_Weight(x)
mapped_compounds["BiGG_ID"]=[model.reactions.get_by_id(compound_id).id[3:] for compound_id in model.medium]
weights=mapped_compounds[["query", "BiGG_ID", "hit", "mol_weight"]]

In [5]:
#outputmodel= cobra.io.save_json_model(model, "Data/iFpraus.json")

In [6]:
#unaltered model
model1=model.copy()
print("\nModel Medium")
model1.objective = {model1.reactions.get_by_id('Biomass_FP'): 1}
print(model1.summary())


Model Medium
Objective
1.0 Biomass_FP = 69.87885590787737

Uptake
------
    Metabolite          Reaction   Flux  C-Number  C-Flux
    Lcystin[e]     EX_Lcystin(e)  3.502         6   0.05%
        ade[e]         EX_ade(e)  864.9         5  10.85%
     adocbl[e]      EX_adocbl(e) 0.3929        72   0.07%
     alaala[e]      EX_alaala(e)  6.462         6   0.10%
     alagln[e]      EX_alagln(e)  30.81         8   0.62%
      arg-L[e]       EX_arg_L(e)   1000         6  15.05%
      asn-L[e]       EX_asn_L(e)   16.4         4   0.16%
        btn[e]         EX_btn(e) 0.3929        10   0.01%
        ca2[e]         EX_ca2(e) 0.3929         0   0.00%
         cl[e]          EX_cl(e) 0.3929         0   0.00%
    cobalt2[e]     EX_cobalt2(e) 0.3929         0   0.00%
     cps_fp[e]      EX_cps_fp(e) 0.3929        63   0.06%
        cu2[e]         EX_cu2(e) 0.3929         0   0.00%
        fe3[e]         EX_fe3(e) 0.7858         0   0.00%
        fol[e]         EX_fol(e)  1.179        19   0.06

### Gifu Anaerobic Medium (mGAM) from:
https://hyserve.com/files/05433_GAM-Broth_Modified_final.pdf


- Peptone, Soya Peptone, Proteose Peptone: source of amino acids -> most likely all of them 
- Yeast extract, liver extract, meat extract, digested serum -> 
    - amino acids, 
    - '4abz(e)' 
    - '5oxpro(e)'
    - thm(e)' thiamin vitamin b1
    - biotin -> btn |from yeast extract? 
    - 'rbflvrd(e)', 'ribflv(e)' vitamin b2
    - 'pydam(e)' pydx pydxn vitamin b6 from yeast extract
    - 'pnto_R(e)' pantothenic acid from yeast extract?
    - 'ptrc(e)' probably from meat extract(e)
    - 'spmd(e)' from meat extract and linked to ptrc
    - cbl1   Cyanocobalamin is a cobalt-containing coordination compound and a natural water-soluble vitamin of the B-complex family 
    - adocbl
    - folate fol is of B vitamin family 
    - tautin from meat extract
    - nac is nicotinate and also vitamin b and so is nacm nicotinamide
    - hxan, gua , ins are all some kind of nucleic acid that is probably present in yeast extract
    - trace elements: Ca2? Cu2? cobalt2?  mg2? pi(e)? so4?
- Dextrose -> same as glucose (https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI:17634) and so -> glc_D
- Soluble starch -> starch1 (potatoe starch)
- L-Tryptophane -> trp__L
- L-Cystein Hydrochloride -> cys_L and cl and h (hydrochloride<-> HCL becomes cl and h)
- Sodium Thioglycolate -> thiog and na1 but it is just a marker for tolerance of oxygen apparently so ignore
- L-Arginine -> arg_L
- Vitamine K1 -> phllqne https://pubchem.ncbi.nlm.nih.gov/compound/Vitamin-K1#section=Depositor-Supplied-Synonyms 
- Hemin -> Iron and Chlorine  fe3 and cl  and perhaps ppp9 https://pubchem.ncbi.nlm.nih.gov/#query=3-%5B18-(2-carboxyethyl)-8%2C13-bis(ethenyl)-3%2C7%2C12%2C17-tetramethylporphyrin-21%2C23-diid-2-yl%5Dpropanoic%20acid%3Biron(3%2B)%3Bchloride 
- Potassium Dihydrogen Phosphate -> pi, k, h
- Sodium Chloride -> na1 cl

Digested serum : https://www.nebiolabs.com.au/-/media/catalog/datacards-or-manuals/p8108datasheet-lot0021306.pdf	

Generally useful page ; https://pubchem.ncbi.nlm.nih.gov/#query=Hemin

check also: https://opencobra.github.io/cobratoolbox/latest/tutorials/tutorialMetabotoolsI.html 

In [7]:
import numpy as np
aa_pre=("ala,arg,asn,asp,asx,cys,glu,gln,glx,gly,his,ile,leu,lys,met,phe,pro,ser,thr,trp,tyr,val").split(",")
aa=[]
for aminoacid in aa_pre:
    if aminoacid != "gly": 
        aa.append(aminoacid +"_L(e)")
    else:
        aa.append(aminoacid+'(e)')
aa=sorted(aa)
aa=np.array(aa)
aa

array(['ala_L(e)', 'arg_L(e)', 'asn_L(e)', 'asp_L(e)', 'asx_L(e)',
       'cys_L(e)', 'gln_L(e)', 'glu_L(e)', 'glx_L(e)', 'gly(e)',
       'his_L(e)', 'ile_L(e)', 'leu_L(e)', 'lys_L(e)', 'met_L(e)',
       'phe_L(e)', 'pro_L(e)', 'ser_L(e)', 'thr_L(e)', 'trp_L(e)',
       'tyr_L(e)', 'val_L(e)'], dtype='<U8')

In [8]:
MediumMgam_complete=[x[3:len(x)] for x in model.medium]
print(MediumMgam_complete)

['3mop(e)', '4abz(e)', '5oxpro(e)', 'Lcyst(e)', 'Lcystin(e)', 'ac(e)', 'acasp(e)', 'acgal(e)', 'acgalglcur(e)', 'acgam(e)', 'acglu(e)', 'acnam(e)', 'ade(e)', 'adocbl(e)', 'ala_D(e)', 'ala_L(e)', 'alaala(e)', 'alaasp(e)', 'alagln(e)', 'alaglu(e)', 'alagly(e)', 'alahis(e)', 'alaleu(e)', 'alathr(e)', 'arab_L(e)', 'arg_L(e)', 'asn_L(e)', 'aso3(e)', 'aso4(e)', 'asp_L(e)', 'btn(e)', 'but(e)', 'butso3(e)', 'ca2(e)', 'cbl1(e)', 'cbl2(e)', 'cd2(e)', 'cellb(e)', 'cgly(e)', 'cit(e)', 'cl(e)', 'co2(e)', 'cobalt2(e)', 'cps_fp(e)', 'cu2(e)', 'cys_L(e)', 'dhor_S(e)', 'ethso3(e)', 'fe2(e)', 'fe3(e)', 'fe3dcit(e)', 'fol(e)', 'for(e)', 'fru(e)', 'fum(e)', 'gal(e)', 'galur(e)', 'gam(e)', 'glc(e)', 'glcur(e)', 'gln_L(e)', 'glu_L(e)', 'gly(e)', 'glyasn(e)', 'glyasp(e)', 'glyc3p(e)', 'glyc_R(e)', 'glygln(e)', 'glyglu(e)', 'glygly(e)', 'glyleu(e)', 'glymet(e)', 'glyphe(e)', 'glypro(e)', 'glytyr(e)', 'gthox(e)', 'gthrd(e)', 'gua(e)', 'h(e)', 'h2o(e)', 'h2s(e)', 'hg2(e)', 'his_L(e)', 'hxan(e)', 'ile_L(e)', 'in

In [12]:
MediumMgam=['4abz(e)','5oxpro(e)','Lcyst(e)', 'Lcystin(e)', 'ade(e)', 'adocbl(e)','ala_D(e)', 'ala_L(e)', 'arg_L(e)', 'asp_L(e)', 'btn(e)', 'ca2(e)', 'cbl1(e)', 'cbl2(e)', 'cl(e)', 'cobalt2(e)', 'cu2(e)', 'cys_L(e)', 'fe2(e)', 'fe3(e)', 'fol(e)', 'glc(e)', 'gln_L(e)', 'glu_L(e)', 'gly(e)',  'gthox(e)', 'gthrd(e)', 'gua(e)', 'h(e)', 'h2o(e)', 'his_L(e)', 'hxan(e)', 'ile_L(e)', 'ins(e)',  'k(e)',  'leu_L(e)', 'lys_L(e)', 'met_D(e)', 'met_L(e)', 'mg2(e)', 'na1(e)', 'nac(e)', 'ncam(e)', 'pb(e)' , 'phe_L(e)', 'pi(e)', 'pnto_R(e)', 'pro_L(e)', 'ptrc(e)', 'pydam(e)', 'pydx(e)', 'pydxn(e)', 'rbflvrd(e)', 'ribflv(e)', 'ser_L(e)', 'so4(e)', 'spmd(e)', 'strch1(e)',  'taur(e)', 'thm(e)', 'thr_L(e)', 'thymd(e)', 'trp_L(e)', 'tyr_L(e)', 'ura(e)', 'val_L(e)', 'xan(e)', 'xyl_D(e)']

In [13]:
#ppm =mg/L =0.001 g/L
#http://www.oxoid.com/UK/blue/prod_detail/prod_detail.asp?pr=LP0037&c=UK&lang=EN&minfo=Y
Traces_BiGG_ID=["ca2(e)" , "cl(e)", "cu2(e)", "fe2(e)", "fe3(e)", "k(e)",  "mg2(e)", "na1(e)","pb(e)"]
conc_ppm_Bact_Peptone_L37=            [635, 1*10*1000, 1, 22/2, 22/2, 3.6*10*1000, 265, 1*10*1000, 0.4]
conc_ppm_Proteose_Peptone_L85=        [200, 8*10*1000, 10, 42/2, 42/2, 1.4*10*1000, 340, 8*10*1000, 0.1]
conc_ppm_soya_peptone_L44=            [225,0.4*10*1000, 2,90/2,90/2, 3.3*10*1000, 1530, 0.4*10*1000, 0.3]
conc_ppm_yeast_extract=               [155,0.3*10*1000, 2,52/2,52/2, 7.3*10*1000, 205, 0.3*10*1000, 0.7]
conc_ppm_Lab_Lemco_L29=               [140,1.1*10*1000, 2,20/2,20/2, 1.9*10*1000, 140, 1.1*10*1000, 0.3] #meat extract
conc_ppm_Liver_Digest_Neutralized_L27=[98,2.2*10*1000, 12,245/2,245/2, 1.6*10*1000, 210, 2.2*10*1000, 0.2] #liver extract

Traces=pd.DataFrame({"BiGG_ID": Traces_BiGG_ID, 
                    "conc_ppm_Bact_Peptone_L37":conc_ppm_Bact_Peptone_L37, 
                    "conc_ppm_Proteose_Peptone_L85":conc_ppm_Proteose_Peptone_L85,
                    "conc_ppm_soya_peptone_L44":conc_ppm_soya_peptone_L44, 
                    "conc_ppm_yeast_extract":conc_ppm_yeast_extract, 
                    'conc_ppm_Lab_Lemco_L29':conc_ppm_Lab_Lemco_L29,
                    'conc_ppm_Liver_Digest_Neutralized_L27' :conc_ppm_Liver_Digest_Neutralized_L27 })

In [14]:
#AA from peptones and yeast extract
#http://www.oxoid.com/UK/blue/prod_detail/prod_detail.asp?pr=LP0037&c=UK&lang=EN&minfo=Y

index_notinPeptones= np.where(np.isin(aa, ['asn_L(e)', 'asx_L(e)','gln_L(e)','glx_L(e)', 'his_L(e)']))
index_notinPeptones=sorted(index_notinPeptones[0], reverse=True)
aa_BiGG_ID=np.delete(aa, index_notinPeptones)

aa_conc_ppm_Bact_Peptone_L37=np.array([3.92,4.99, 6.06, 1.66, 9.93, 7.71, 3.81, 3.79, 4.38, 1.58, 2.60, 5.83, 2.81, 1.25, 0.66, 0.39, 3.33])*10*1000
aa_conc_ppm_Proteose_Peptone_L85=np.array([3.81, 5.80, 5.85, 0.15, 13.78, 4.45, 4.58, 6.01, 4.61, 1.08, 4.66, 5.99, 2.18, 2.75, 0.75, 1.77, 4.11])*10*1000
aa_conc_ppm_soya_peptone_L44=np.array([2.57, 4.64, 7.06, 0.53, 14.71, 2.83, 2.51, 4.31, 3.77, 0.62, 0.38, 3.40, 0.67, 1.68, 0.64, 2.09, 3.65])*10*1000
aa_conc_Yeast_Extract_L21=np.array([0.91, 3.31, 7.07, 0.76, 13.49, 5.95, 4.81, 6.04, 5.40, 0.80, 3.78, 0.88, 3.42, 2.73, 0.85, 4.95, 1.00])*10*1000
aa_conc_ppm_Lab_Lemco_L29=np.array([5.85, 7.10, 5.10, 0.68, 10.71, 10.85, 3.17, 3.15, 4.78, 2.61, 2.34, 7.79, 1.87, 2.54, 0.34, 0.66, 3.06 ])*10*1000
aa_conc_ppm_Liver_Digest_Neutralized_L27=np.array([3.40, 2.89, 5.85, 0.34, 10.38, 3.83, 2.11, 5.96, 6.02, 0.95, 2.45, 2.38, 0.83, 1.42, 1.02, 2.54, 3.05 ])*10*1000

AA=pd.DataFrame({"BiGG_ID": aa_BiGG_ID, 
                    "conc_ppm_Bact_Peptone_L37":aa_conc_ppm_Bact_Peptone_L37,
                    "conc_ppm_Proteose_Peptone_L85":aa_conc_ppm_Proteose_Peptone_L85,
                    "conc_ppm_soya_peptone_L44":aa_conc_ppm_soya_peptone_L44,
                    "conc_ppm_yeast_extract":aa_conc_Yeast_Extract_L21,  
                    'conc_ppm_Lab_Lemco_L29':aa_conc_ppm_Lab_Lemco_L29,
                    'conc_ppm_Liver_Digest_Neutralized_L27' : aa_conc_ppm_Liver_Digest_Neutralized_L27 })


In [15]:
Weights_Mgam=weights[weights["BiGG_ID"].isin(MediumMgam)]
Traces_AA_ppm=AA.append(Traces)
Traces_AA_ppm=pd.merge(Traces_AA_ppm, Weights_Mgam, on='BiGG_ID')

In [16]:
col_list= list(Traces_AA_ppm)
col_list.remove('mol_weight')

['BiGG_ID',
 'conc_ppm_Bact_Peptone_L37',
 'conc_ppm_Proteose_Peptone_L85',
 'conc_ppm_soya_peptone_L44',
 'conc_ppm_yeast_extract',
 'conc_ppm_Lab_Lemco_L29',
 'conc_ppm_Liver_Digest_Neutralized_L27',
 'query',
 'hit']

In [18]:
if "sum_ppm" not in Traces_AA_ppm.columns:
    Traces_AA_ppm["sum_ppm"]=Traces_AA_ppm[col_list].sum(numeric_only=True, axis=1)
Traces_AA_ppm["sum_mg_L"]=Traces_AA_ppm["sum_ppm"]
Traces_AA_ppm["sum_mg_L"]=np.array(Traces_AA_ppm["sum_ppm"])
Traces_AA_ppm["g_L"]=np.array(Traces_AA_ppm["sum_ppm"])/1000
Traces_AA_ppm

,BiGG_ID,conc_ppm_Bact_Peptone_L37,conc_ppm_Proteose_Peptone_L85,conc_ppm_soya_peptone_L44,conc_ppm_yeast_extract,conc_ppm_Lab_Lemco_L29,conc_ppm_Liver_Digest_Neutralized_L27,query,hit,mol_weight,sum_ppm,sum_mg_L,g_L
0,ala_L(e),39200.0,38100.0,25700.0,9100.0,58500.0,34000.0,L-Alanine,L-Alanine,89.09322,204600.0,204600.0,204.6000
1,arg_L(e),49900.0,58000.0,46400.0,33100.0,71000.0,28900.0,L-Arginine,L-Arginine,174.20100,287300.0,287300.0,287.3000
2,asp_L(e),60600.0,58500.0,70600.0,70700.0,51000.0,58500.0,L-Aspartate,L-Aspartic acid,133.10270,369900.0,369900.0,369.9000
3,cys_L(e),16600.0,1500.0,5300.0,7600.0,6800.0,3400.0,L-Cysteine,L-Cysteine,121.15800,41200.0,41200.0,41.2000
4,glu_L(e),99300.0,137800.0,147100.0,134900.0,107100.0,103800.0,L-Glutamate,L-Glutamic acid,147.12930,730000.0,730000.0,730.0000
5,gly(e),77100.0,44500.0,28300.0,59500.0,108500.0,38300.0,Glycine,Glycine,75.06664,356200.0,356200.0,356.2000
6,ile_L(e),38100.0,45800.0,25100.0,48100.0,31700.0,21100.0,L-Isoleucine,L-Isoleucine,131.17500,209900.0,209900.0,209.9000
7,leu_L(e),37900.0,60100.0,43100.0,60400.0,31500.0,59600.0,L-Leucine,L-Leucine,131.17296,292600.0,292600.0,292.6000
8,lys_L(e),43800.0,46100.0,37700.0,54000.0,47800.0,60200.0,L-Lysine,L-Lysine,146.18764,289600.0,289600.0,289.6000
9,met_L(e),15800.0,10800.0,6200.0,8000.0,26100.0,9500.0,L-Methionine,L-Methionine,149.21238,76400.0,76400.0,76.4000


In [19]:
#Defined ingredients
defined=np.array(['glc(e)','strch1(e)', 'trp_L(e)', 'cys_L(e)', 'h(e)', 'cl(e)', 'na1(e)', 'arg_L(e)', 'k(e)', 'pi(e)'])
conc_defined_g_l=np.array([0.5, 5, 0.2, 0.3, 0.3, 0.3+3, 0.3+3, 1, 2.5, 2.5])
defined_g_L=pd.merge(pd.DataFrame({'BiGG_ID':defined, 'g_L':conc_defined_g_l}), Weights_Mgam, on='BiGG_ID')
defined_g_L

,BiGG_ID,g_L,query,hit,mol_weight
0,glc(e),0.5,D-Glucose,D-Glucose,180.15588
1,strch1(e),5.0,"starch, structure 1 (1,6-{7[1,4-Glc], 4[1,4-Gl...",,NaN
2,trp_L(e),0.2,L-Tryptophan,L-Tryptophan,204.22526
3,cys_L(e),0.3,L-Cysteine,L-Cysteine,121.15800
4,h(e),0.3,H,L-Histidine,155.15468
5,cl(e),3.3,Chloride,Chloride ion,35.45270
6,na1(e),3.3,Sodium,Sodium,22.98977
7,arg_L(e),1.0,L-Arginine,L-Arginine,174.20100
8,k(e),2.5,K,L-Lysine,146.18764
9,pi(e),2.5,Phosphate,Phosphate,97.99520


In [20]:
Traces_AA_def=Traces_AA_ppm[["BiGG_ID", "query", "hit", "g_L", "mol_weight"]].append(defined_g_L).reset_index()

missing_weights_BiGG_ID=np.array(Traces_AA_def[Traces_AA_def['mol_weight'].isnull()]["BiGG_ID"])
missing_weights=Mol_Weight(["Copper", "Fe", "Magnesium", "Maltoundecadose"]) #Maltoundecadose equals 
Fe=missing_weights.loc[1:1]
missing_weights_weights = pd.concat([missing_weights.loc[:1], Fe, missing_weights.loc[2:]]).reset_index(drop=True)
missing_weights_weights.loc[4:4]["mol_weight"]=1801.6 #g/mol https://pubchem.ncbi.nlm.nih.gov/compound/Maltoundecaose#section=Computed-Properties 

In [21]:
index=Traces_AA_def[Traces_AA_def["mol_weight"].isnull()].index
for i in range(0,len(index)):
    Traces_AA_def.loc[index[i],"mol_weight"]= missing_weights_weights["mol_weight"][i]
Traces_AA_def["mmol/L"]=Traces_AA_def["g_L"]/Traces_AA_def["mol_weight"]*1000
Traces_AA_def

,index,BiGG_ID,query,hit,g_L,mol_weight,mmol/L
0,0,ala_L(e),L-Alanine,L-Alanine,204.6000,89.09322,2296.471045
1,1,arg_L(e),L-Arginine,L-Arginine,287.3000,174.20100,1649.244264
2,2,asp_L(e),L-Aspartate,L-Aspartic acid,369.9000,133.10270,2779.057074
3,3,cys_L(e),L-Cysteine,L-Cysteine,41.2000,121.15800,340.051833
4,4,glu_L(e),L-Glutamate,L-Glutamic acid,730.0000,147.12930,4961.622192
5,5,gly(e),Glycine,Glycine,356.2000,75.06664,4745.117139
6,6,ile_L(e),L-Isoleucine,L-Isoleucine,209.9000,131.17500,1600.152468
7,7,leu_L(e),L-Leucine,L-Leucine,292.6000,131.17296,2230.642657
8,8,lys_L(e),L-Lysine,L-Lysine,289.6000,146.18764,1981.015632
9,9,met_L(e),L-Methionine,L-Methionine,76.4000,149.21238,512.021858


In [22]:
MediumMgam=['4abz(e)','5oxpro(e)','Lcyst(e)', 'Lcystin(e)', 'ade(e)', 'adocbl(e)','ala_D(e)', 'ala_L(e)', 'arg_L(e)', 'asp_L(e)', 'btn(e)', 'ca2(e)', 'cbl1(e)', 'cbl2(e)', 'cl(e)', 'cobalt2(e)', 'cu2(e)', 'cys_L(e)', 'fe2(e)', 'fe3(e)', 'fol(e)', 'glc(e)', 'gln_L(e)', 'glu_L(e)', 'gly(e)',  'gthox(e)', 'gthrd(e)', 'gua(e)', 'h(e)', 'h2o(e)', 'his_L(e)', 'hxan(e)', 'ile_L(e)', 'ins(e)',  'k(e)',  'leu_L(e)', 'lys_L(e)', 'met_D(e)', 'met_L(e)', 'mg2(e)', 'na1(e)', 'nac(e)', 'ncam(e)', 'pb(e)' , 'phe_L(e)', 'pi(e)', 'pnto_R(e)', 'pro_L(e)', 'ptrc(e)', 'pydam(e)', 'pydx(e)', 'pydxn(e)', 'rbflvrd(e)', 'ribflv(e)', 'ser_L(e)', 'so4(e)', 'spmd(e)', 'strch1(e)',  'taur(e)', 'thm(e)', 'thr_L(e)', 'thymd(e)', 'trp_L(e)', 'tyr_L(e)', 'ura(e)', 'val_L(e)', 'xan(e)', 'xyl_D(e)']

model_mgam=medium(model.copy(),MediumMgam)
print('\nMedium_mgam')
for j,i in enumerate(Traces_AA_def["BiGG_ID"]):
    model_mgam.exchanges.get_by_id("EX_"+i).lower_bound=-Traces_AA_def.loc[j,"mmol/L"]
model_mgam.summary()


Medium_mgam


Metabolite,Reaction,Flux,C-Number,C-Flux
Lcystin[e],EX_Lcystin(e),0.5354,6,0.08%
adocbl[e],EX_adocbl(e),0.06007,72,0.10%
ala-D[e],EX_ala_D(e),1.976,3,0.14%
ala-L[e],EX_ala_L(e),6.33,3,0.46%
arg-L[e],EX_arg_L(e),5.388,6,0.78%
btn[e],EX_btn(e),0.06007,10,0.01%
ca2[e],EX_ca2(e),0.06007,0,0.00%
cl[e],EX_cl(e),0.06007,0,0.00%
cobalt2[e],EX_cobalt2(e),0.06007,0,0.00%
cu2[e],EX_cu2(e),0.06007,0,0.00%


In [67]:
MediumMgam=['4abz(e)','5oxpro(e)','Lcyst(e)', 'Lcystin(e)', 'ade(e)', 'adocbl(e)','ala_D(e)', 'ala_L(e)', 'arg_L(e)', 'asp_L(e)', 'btn(e)', 'ca2(e)', 'cbl1(e)', 'cbl2(e)', 'cl(e)', 'cobalt2(e)', 'cu2(e)', 'cys_L(e)', 'fe2(e)', 'fe3(e)', 'fol(e)', 'glc(e)', 'gln_L(e)', 'glu_L(e)', 'gly(e)',  'gthox(e)', 'gthrd(e)', 'gua(e)', 'h(e)', 'h2o(e)', 'his_L(e)', 'hxan(e)', 'ile_L(e)', 'ins(e)',  'k(e)',  'leu_L(e)', 'lys_L(e)', 'met_D(e)', 'met_L(e)', 'mg2(e)', 'na1(e)', 'nac(e)', 'ncam(e)', 'pb(e)' , 'phe_L(e)', 'pi(e)', 'pnto_R(e)', 'pro_L(e)', 'ptrc(e)', 'pydam(e)', 'pydx(e)', 'pydxn(e)', 'rbflvrd(e)', 'ribflv(e)', 'ser_L(e)', 'so4(e)', 'spmd(e)', 'strch1(e)',  'taur(e)', 'thm(e)', 'thr_L(e)', 'thymd(e)', 'trp_L(e)', 'tyr_L(e)', 'ura(e)', 'val_L(e)', 'xan(e)', 'xyl_D(e)']

comparison=pd.DataFrame({"Compound":MediumMgam, "Specific_Bound_Set":pd.Series(MediumMgam).isin(Traces_AA_def["BiGG_ID"])})
no_specific_bound=comparison.loc[comparison["Specific_Bound_Set"]==False]
len(no_specific_bound), len(MediumMgam)

(38, 68)

In [ ]:
for i in MediumMgam:
    print(model_mgam.reactions.get_by_id(i))
    print(model_mgam.reactions.get_by_id(i).lower_bound)

In [ ]:
find_match(model_mgam,MediumMgam)

In [ ]:
for i in find_match(MediumMgam,)[1]:
    r = re.compile(r'{0}.+?{1}?.+?{2}?.*?'.format(i[0:5],i[5], i[6]))
    newlist = list(filter(r.findall, x)) # Read Note
    print(newlist)

In [ ]:
fba = model.optimize()
fluxes = deleteNull(fba.fluxes)
print('\nActive reaction : %i which represent %f%% of the model' %(len(fluxes),len(fluxes)/len(model.reactions)*100))
print(fba.objective_value)

In [ ]:
from cobra.medium import minimal_medium

max_growth = model.slim_optimize()
minimal_medium(model, 0.005)

In [ ]:
from cobra.medium import minimal_medium
max_growth = model_mgam.slim_optimize()
minimal_medium(model_mgam, max_growth)

---
## Reinforced Clostridial Medium (RCM)
http://www.oxoid.com/UK/blue/prod_detail/prod_detail.asp?pr=CM0149&org=53&c=UK&lang=EN

* Yeast Extract -> [lots of stuff in it, unsure of what's garbage](https://www.chemicalbook.com/ChemicalProductProperty_EN_CB9440339.htm#:~:text=Yeast%20extract%20has%20a%20protein,aromatic%20compounds%20and%20other%20components.)
    * Glutathione: gthrd (reduced version)
    * 18 AA's: see above
    * Dextran: glc__D (it's made of glucose)
    * Mannan: mannan
    * Trehalose: tre
    * B-vitamins: (Clemence mentioned vitamins being important so I'm putting them all in)
        * \1. Thiamin -> thm
        * \2. Riboflavin -> ribflv
        * \3. Niacin -> trp__L (made of tryptophan)
        * \5. Pantothenic Acid -> pnto__R
        * \6. Pyridoxine -> pydxn
        * \7. Biotin -> btn
        * \9. Folic Acid -> ... can't find anything
        * \12. Cobalamin -> b12 (or cbl1)
    * Biotin
* Peptone -> Sticking with the above logic, probably a source of all AAs
* Glucose -> Assuming D_Glucose: glc__D
* Soluble Starch -> starch1200
* Sodium Chloride -> na1, cl
* Sodium Acetate -> Assuming breaking down into sodium and acetate: ac, na1
* Cysteine Hydrochloride -> cys_L, cl, h (could also be cys_D? cys_L is more widely used)
* Agar: Agarose (70%) and Agaropectin (30%) [according to Wiki](https://en.wikipedia.org/wiki/Agar)
    * Agarose: D-galactose & 3,6-anhydro-L-galactopyranose
        * D-galactose: gal
        * 3,6-anhydro-L-galactopyranose: [similar structure & compositon to beta-D-allose](https://pubchem.ncbi.nlm.nih.gov/#query=CID67020466%20structure&tab=similarity)
            * D-Allose: all__D
    * Agaropectin: D-galactose, L-galactose, pyruvate, sulfate
        * D-galactose: gal
        * L-galactose: gal__L
        * Pyruvate: pyr
        * Sulfate: so4

| Component              | Concentration (g/L) | BiGG Metabolites                                                                          |
|------------------------|---------------------|-------------------------------------------------------------------------------------------|
| Yeast Extract          | 13.0                | (see all AA's), gthrd, glc\__D, mannan, tre, thm, ribflv, trp\__L, pnto\__R, pydxn, btn, b12 |
| Peptone                | 10.0                | (see all AA's)                                                                            |
| Glucose                | 5.0                 | glc_D                                                                                     |
| Soluble Starch         | 1.0                 | starch1200                                                                                |
| Sodium Chloride        | 5.0                 | na1, cl                                                                                   |
| Sodium Acetate         | 3.0                 | na1, ac                                                                                   |
| Cysteine Hydrochloride | 0.5                 | cys_L, cl, h                                                                              |
| Agar                   | 0.5                 | gal, all\__D, gal\__L, pyr, so4                                                           |

In [ ]:
medium_rcm = list(('gthrd','glc__D','mannan','tre','thm','ribflv','trp__L','pnto__R','pydxn','btn','b12',
             'starch1200','na1','ac','cl','cys_L','gal','all__D','gal__L','pyr','so4'))
medium_rcm=[mets+"(e)" for mets in medium_rcm]
model_rcm=medium(model.copy(), medium_rcm)
for i in range(1,len(model_rcm.reactions)):
    model_rcm.reactions[i].upper_bound=1000
    model_rcm.reactions[i].lower_bound=-1000
model_rcm.objective = {model_rcm.reactions.get_by_id('Biomass_FP'): 1}
model_rcm.summary()

In [ ]:
model_mmcb.reactions.get_by_id('mannan')

In [ ]:
# Post-optimization
fba_rcm = model_rcm.optimize()
fluxes = deleteNull(fba_rcm.fluxes)
print('\nActive reaction : %i which represent %f%% of the model' %(len(fluxes),len(fluxes)/len(model.reactions)*100))
print(model_rcm.objective.value)

---
## mMCB

(There are also supplements mentioned in the paper but this is the base medium)
* [Bacteriological Peptome (6.5) (Oxoid)](http://www.oxoid.com/UK/blue/prod_detail/prod_detail.asp?pr=LP0037&c=UK&lang=EN): 
    * Vague, decided to go with polypeptides: <b>polypep</b>
    * Nitrogen: <b>n2</b>
* [Soy Peptome (5.0) (Oxoid)](http://www.oxoid.com/UK/blue/prod_detail/prod_detail.asp?pr=LP0044&cat=&c=UK&lang=EN) --> another vague one:
    * Stachyose: <b>stys</b>
    * Raffinose: <b>raffin</b>
    * Sucrose: <b>sucr</b>
    * Nitrogen: <b>n2</b>
* [Yeast Extract (3.0) (VWR International, Darmstadt, Germany)](https://us.vwr.com/store/product/7437401/vwr-life-science-yeast-extract-bacteriological-grade):
    * Vitamin B (same logic as RCM):
        * 1. Thiamin -> <b>thm</b>
        * 2. Riboflavin -> <b>ribflv</b>
        * 3. Niacin -> <b>trp__L</b> (made of tryptophan)
        * 5. Pantothenic Acid -> <b>pnto__R</b>
        * 6. Pyridoxine -> <b>pydxn</b>
        * 7. Biotin -> <b>btn</b>
        * 9. Folic Acid -> ... can't find anything
        * 12. Cobalamin -> <b>b12</b> (or cbl1)
* [Tryptone (2.5) (Oxoid)](http://www.oxoid.com/UK/blue/prod_detail/prod_detail.asp?pr=LP0042&c=UK&lang=EN): 
    * Tryptophan: <b>trp__L</b>
* NaCL (1.5) (VWR International, Darmstadt, Germany):
    * NaCl: <b>na1, cl</b>
* K<sub>2</sub>HPO<sub>4</sub> (1.0) (Merck International, Darmstadt, Germany):
    * Potassium: <b>k</b>
    * Phosphate: <b>p1</b>
    * Hydrogen: <b>h2</b>
* KH<sub>2</sub>PO<sub>4</sub> (1.0) (Merck International, Darmstadt, Germany):
    * Potassium: <b>k</b>
    * Phosphate: <b>p1</b>
    * Hydrogen: <b>h2</b>
* Na<sub>2</sub>SO<sub>4</sub> (2.0) (VWR):
    * Sodium: <b>na1</b>
    * Sulfate: <b>so4</b>
* MgSO<sub>4</sub>*7H<sub>2</sub>O (1.0) (Merck):
    * Magnesium: <b>mg2</b>
    * Sulfate: <b>so4</b>
* CaCl<sub>2</sub>*2H<sub>2</sub>O (0.1) (Merck):
    * Calcium Chloride: <b>ca2, cl</b>
* NH<sub>4</sub>Cl (1.0) (Merck):
    * Ammonium Chloride: <b>nh4, cl</b>
* Cysteine-HCL (0.4) (Merck):
    * Cysteine: <b>cys__L</b>
    * HCL: <b>h2, cl</b>
* NaHCO<sub>3</sub> (0.2) (VWR):
    * Sodium: <b>na1</b>
    * Bicarbonate: <b>hco3<b>
* MnSO<sub>4</sub>*H<sub>2</sub>O (0.05) (VWR):
    * Manganese: <b>mn2</b>
    * Sulfate: <b>so4</b>
* FeSO<sub>4</sub>*7H<sub>2</sub>O (0.005) (Merck):
    * Iron: <b>fe</b>
    * Sulfate: <b>so4</b>
* ZnSO<sub>4</sub>*7H<sub>2</sub>O (0.005) (VWR):
    * Zinc: <b>zn2</b>
    * Sulfate: <b>so4</b>
* Hemin (0.005) (Sigma-Aldrich, Steinheim, Germany):
    * ... contains iron
* Menadione (0.005) (S-A):
    * <b>mndn</b>
* Resazurin (0.001) (S-A):
    * Fluoro identifier



| Component                            | Concentration (g/L) | BiGG Metabolites                                |
|--------------------------------------|---------------------|-------------------------------------------------|
| Bacteriological Peptome              | 6.5                 | polypep, n2                                     |
| Soy Peptome                          | 5.0                 | stys, raffin, sucr, n2                          |
| Yeast Extract                        | 3.0                 | thm, ribflv, trp\__L, pnto\__R, pydxn, btn, b12 |
| Tryptone                             | 2.5                 | trp\__L                                         |
| NaCL                                 | 1.5                 | na1, cl                                         |
| K<sub>2</sub>PO<sub>4</sub>          | 1.0                 | k, p1, h2                                       |
| KH<sub>2</sub>PO<sub>4</sub>         | 1.0                 | k, p1, h2                                       |
| Na<sub>2</sub>SO<sub>4</sub>         | 2.0                 | na1, so4                                        |
| Mg<sub>2</sub>SO<sub>4</sub>         | 1.0                 | mg2, so4                                        |
| CaCl<sub>2</sub>*2H<sub>2</sub>O     | 0.1                 | ca2, cl                                         |
| NH<sub>4</sub>Cl                     | 1.0                 | nh4, cl                                         |
| Cysteine-HCL                         | 0.4                 | cys__L, h2, cl                                  |
| NaHCO<sub>3</sub>                    | 0.2                 | na1, hco3                                       |
| MnSO<sub>4</sub>*H<sub>2</sub>O      | 0.05                | mn2, so4                                        |
| FeSOMnSO<sub>4</sub>*7H<sub>2</sub>O | 0.005               | fe, so4                                         |
| ZnSO<sub>4</sub>*7H<sub>2</sub>O     | 0.005               | zn2, so4                                        |
| Hemin                                | 0.005               |                                                 |
| Menadione                            | 0.005               | mndn                                            |
| Resazurin                            | 0.001               |                                                 |

In [ ]:
model.copy()

In [ ]:
medium_mmcb = list(('polypep','n2','stys','raffin','sucr','n2','thm','ribflv','trp__L',
                    'pnto__R','pydxn','btn','b12','trp__L','na1','cl','k','p1','h2','na1',
                    'so4','mg2','ca2','nh4','cys__L','h2','cl','na1','hco3','mn2','zn2','mndnb'))

In [ ]:
medium_mmcb=[mets+"(e)" for mets in medium_mmcb]
model_mmcb=medium(model.copy(), medium_mmcb)
unused = []
for i in range(1,len(model_mmcb.reactions)):
    model_mmcb.reactions[i].upper_bound=1000
    if model_mmcb.reactions[i].lower_bound != 0:
        model_mmcb.reactions[i].lower_bound=-10
    else:    
        model_mmcb.reactions[i].lower_bound=-
model_mmcb.objective = {model_mmcb.reactions.get_by_id('Biomass_FP'): 1}
model_mmcb.summary()

# Find list of all metabolites not in media
# Express in mmol/g/L
# Set as -'ve flux lower bound

In [ ]:
# Post-optimization
fba_mmcb = model_mmcb.optimize()
fluxes_mmcb = deleteNull(fba_mmcb.fluxes)
print('\nActive reaction : %i which represent %f%% of the model' %(len(fluxes_mmcb),len(fluxes_mmcb)/len(model.reactions)*100))
print(model_mmcb.objective.value)